<a href="https://colab.research.google.com/github/pat-hare/DeepLearningFashionMNist/blob/main/PatrickH_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
# Setting up google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')

Mounted at /content/gdrive


In [ ]:
import my_utils as mu
import torch
import helper
import matplotlib.pyplot as plt
import numpy as np
from torch import nn
from torch.optim import SGD, Adam
import torch.nn.functional as F
from IPython import display
from tqdm.notebook import trange, tqdm

import copy
import random
import time

# Load Data

In [ ]:
# during dev, using batch size of 100 images
batch_size = 256
trainset, testset = mu.load_data_fashion_mnist(batch_size)

trainset

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# The Stem

In [ ]:
image, label = next(iter(trainset))

# Take input of image I of size HxW (images from Fashion MNist are 28x28) and convert to 4 x (14x14)
def stem(image):
  # Divide into Np patches of dimension KxK (i.e. 4 patches of 14x14)
  kc, kh, kw = 1, 14, 14
  dc, dh, dw = 14, 14, 14

  patches = image.unfold(1, kc, dc).unfold(2, kh, dh).unfold(3, kw, dw)
  unfold_shape = patches.size()
  patches.contiguous().view(-1, kc, kh, kw)

  return np.reshape(patches, (4,196))

x = stem(image)

print(x)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


ValueError: ignored

# The Backbone

In [ ]:
class Net(nn.Module):
  def __init__(self, input_size=784, output_size=10, layers=[196,84]):
    super().__init__()
    # use stem
    self.d1 = nn.Linear(input_size, 196) # stem 784 -> 196
    self.d2 = nn.Linear(196, 64)
    self.d3 = nn.Linear(64, output_size)

    torch.nn.init.normal_(self.d1.weight, std=0.01)
    torch.nn.init.zeros_(self.d1.bias)

    torch.nn.init.normal_(self.d2.weight, std=0.01)
    torch.nn.init.zeros_(self.d2.bias)

    torch.nn.init.normal_(self.d3.weight, std=0.01)
    torch.nn.init.zeros_(self.d3.bias)

  def forward(self, x):
        # x = [batch size, height, width]
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)
        # x = [batch size, height * width]

        # g(XtW1) W2, g = relu, W1 d1.weight, W2 d1.bias
        h_1 = F.relu(self.d1(x))

        # h_1 = [batch size, 250]

        h_2 = F.relu(self.d2(h_1))

        # h_2 = [batch size, 100]

        y_pred = self.d3(h_2)

        # y_pred = [batch size, output dim]

        return y_pred, h_2

model = Net()
print(model)

criterion = nn.CrossEntropyLoss()
optimiser = Adam(model.parameters())

Net(
  (d1): Linear(in_features=784, out_features=196, bias=True)
  (d2): Linear(in_features=196, out_features=64, bias=True)
  (d3): Linear(in_features=64, out_features=10, bias=True)
)


In [ ]:
# input_size = 784 #i.e 28*28*1 
# hidden_size = [128,64,32]
# out_size = 10

class Net(nn.Module):
  def __init__(self, input_size=784, output_size=10, hidden_size=[128,64,32]):
    super().__init__()
    # use stem for intital patch creation here? 

    self.linear1 = nn.Linear(input_size, hidden_size[0]) #MLP1 input layer
    self.linear2 = nn.Linear(hidden_size[0], hidden_size[0]) #MLP1

    self.linear3 = nn.Linear(hidden_size[0], hidden_size[1]) #MLP2 input layer
    self.linear4 = nn.Linear(hidden_size[1], output_size) #MLP2

  #initializing the weights
    torch.nn.init.normal_(self.linear1.weight, std=0.01)
    torch.nn.init.zeros_(self.linear1.bias)

    torch.nn.init.normal_(self.linear2.weight, std=0.01)
    torch.nn.init.zeros_(self.linear2.bias)

    torch.nn.init.normal_(self.linear3.weight, std=0.01)
    torch.nn.init.zeros_(self.linear3.bias)

  def forward(self, x):
        batch_size = x.shape[0]
        x = x.view(batch_size, -1)

        # g(XtW1)W2, g = relu 
        h_1 = F.relu(self.linear1(x))  # Apply activation function - ReLU to the first MLP
        h_2 = self.linear2(h_1)
        h_3 = F.relu(self.linear3(h_2))  # Apply activation function - ReLU to the second MLP
        y_pred = self.linear4(h_3)

   #softmax regression classifier here?

        return y_pred, h_3

model = Net()
print(model)


criterion = nn.CrossEntropyLoss()
optimiser = Adam(model.parameters())

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for (x, y) in tqdm(iterator, desc="Training", leave=False):

        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        # vectorise + stem
        y_pred, _ = model(x)

        loss = criterion(y_pred, y)

        acc = calculate_accuracy(y_pred, y)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for (x, y) in tqdm(iterator, desc="Evaluating", leave=False):

            x = x.to(device)
            y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
EPOCHS = 10

best_valid_loss = float('inf')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for epoch in trange(EPOCHS):

    start_time = time.monotonic()

    train_loss, train_acc = train(model, trainset, optimiser, criterion, device)
    valid_loss, valid_acc = evaluate(model, testset, criterion, device)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  0%|          | 0/10 [00:00<?, ?it/s]

Training:   0%|          | 0/235 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 0.175 | Train Acc: 93.51%
	 Val. Loss: 0.315 |  Val. Acc: 89.96%


...

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch: 09 | Epoch Time: 0m 8s
	Train Loss: 0.148 | Train Acc: 94.54%
	 Val. Loss: 0.339 |  Val. Acc: 89.50%


Training:   0%|          | 0/235 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Epoch: 10 | Epoch Time: 0m 8s
	Train Loss: 0.145 | Train Acc: 94.63%
	 Val. Loss: 0.338 |  Val. Acc: 89.59%


# HIDE

In [ ]:
# Consists of N blocks
# Minimum impl of each block is 2 MLPs
# MLP1 => O1 = g(XtW1)W2, g is a non-linear activation function; then O1 <- OT1
# MLP2 => O2 = g(O1W3)W4

# Other components can be added based on W5-8
# input_size = 784 #i.e 28*28*1 
# hidden_size = [256,128,64]
# out_size = 10


# model = nn.Sequential(
#     nn.Linear(input_size, hidden_size[0]),
#     nn.ReLU(),
#     nn.Linear(hidden_size[0], hidden_size[1]),
#     nn.ReLU(),
#     nn.Linear(hidden_size[1], hidden_size[2]),
#     nn.ReLU(),
#     nn.Linear(hidden_size[2],out_size),
#     nn.LogSoftmax(dim=1)
# )

# criterion = nn.NLLLoss()
# optimiser = SGD(model.parameters(),lr=0.001)

In [ ]:
epochs =10

for e in range(epochs):
  running_loss = 0
  for images, labels in train_iter:
    
    #Flatten the image into a 784 long vector
    
    images = images.view(images.shape[0],-1) #sqash the image in to 784*1 vector
    
    #reset the default gradients
    optimiser.zero_grad()
    
    # forward pass
    output = model(images)
    loss = criterion(output,labels)
    
    #backward pass calculate the gradients for loss
    loss.backward()
    
    # update the parameters
    optimiser.step()
    
    running_loss = running_loss+loss.item()
  else:
    print(f"Training loss: {running_loss/len(train_iter)}")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training loss: 2.3007224656767766
Training loss: 2.2986847202656633
Training loss: 2.2965006787898177
Training loss: 2.2944956755234025
Training loss: 2.29239060110965
Training loss: 2.2901454032477684
Training loss: 2.2879444421347923
Training loss: 2.2856951810545842
Training loss: 2.283212174803524
Training loss: 2.2806997339604265


# Loss

# Optimizer

# The Classifier

In [ ]:
# Takes O2 of the last block
# Computes a mean feature 
# Feeds into a classifier
# Can be softmax regression classifier

# Training

In [ ]:
epochs = 10
train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
  trn_corr = 0
  tst_corr = 0

  for b, (X_train, y_train) in enumerate(trainset):
    b += 1

    y_pred = model(X_train.view(100,-1))
    loss = criterion(y_pred, y_train)

    predicted = torch.max(y_pred.data, 1)[1]
    batch_corr = (predicted == y_train).sum()
    trn_corr += batch_corr

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

  train_losses.append(loss)
  train_correct.append(trn_corr)

  with torch.no_grad():
    for b, (X_test, y_test) in enumerate(testset):
      y_val = model(X_test.view(100,-1))

      predicted = torch.max(y_val.data, 1)[1]
      tst_corr += (predicted == y_test).sum()

  loss = criterion(y_val, y_test)
  test_losses.append(loss)
  test_correct.append(tst_corr)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


RuntimeError: ignored

In [ ]:
print(test_correct[-1].item()*100/10000)

88.44
